In [ ]:
import psycopg2 as pg
import psycopg2.extras as extras 
import pandas as pd
import numpy as np
import tiktoken
import os
from pgvector.psycopg2 import register_vector
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

conn = pg.connect(os.getenv('AI_DB_URL'))
open_ai_key = os.environ["OPENAI_API_KEY"]

# commit history column headers
col_headers = ['id', 'author', 'commit_time', 'commit_id', 'commit_title', 'commit_comments']

# tiktoken encoding: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
tt_encoding = 'cl100k_base'

# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricin
embed_cost = 0.0000001 # $0.10 per 1M  <check current pricing!>

# choose from dictionary models directly below
model_num = 26 

# from running config_openai.ipynb
models = {  0: 'gpt-4o-2024-05-13', 1: 'gpt-4o', 2: 'gpt-4-turbo-2024-04-09', 3: 'gpt-4-turbo', 4: 'gpt-3.5-turbo-0125',
            5: 'gpt-4-turbo-preview', 6: 'gpt-4-0125-preview', 7: 'text-embedding-3-large', 8: 'text-embedding-3-small',
            9: 'tts-1-hd-1106', 10: 'tts-1-1106', 11: 'tts-1-hd', 12: 'gpt-3.5-turbo-1106', 13: 'gpt-4-1106-preview',
            14: 'dall-e-2', 15: 'dall-e-3', 16: 'gpt-3.5-turbo-instruct-0914', 17: 'gpt-3.5-turbo-instruct',
            18: 'babbage-002', 19: 'davinci-002', 20: 'gpt-4', 21: 'gpt-4-0613', 22: 'gpt-3.5-turbo-16k',
            23: 'tts-1', 24: 'gpt-3.5-turbo', 25: 'whisper-1', 26: 'text-embedding-ada-002'  }

# Helper func: calculate number of tokens
def num_tokens_from_string(string, encoding_name):
    try:
        if not string:
            return False # maybe raise exception instead :)
        # Returns the number of tokens in a text string
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens
    except Exception as e:
            print('Error in num_tokens_from_string: {}'.format(e))   

# Helper function: calculate cost of embedding num_tokens        
def get_embedding_cost(num_tokens, token_cost):
    try:
        return num_tokens * token_cost
    except Exception as e:
        print('Error in num_tokens_from_string: {}'.format(e))      

class Openai:
    def __init__(self):
        self.ch_columns = col_headers
        self.encoding = tt_encoding
        self.embed_cost = embed_cost
        self.embed_model = models[model_num]
        
    def get_params(self):
        return {
            'ch_cols': self.ch_columns,
            'encoding': self.encoding,
            'token_cost': self.embed_cost,
            'model': self.embed_model
        }
    
    def get_notices(self):
        for notice in conn.notices:
            print('')
            print('***Message from Postgres: {}'.format(notice))
        return True
    
    def get_total_embeddings_cost(self, old_df, encode, cost):
        try:
            df = old_df.copy(deep=True)
            df['content'] = df['author'] + df['commit_time'] + df['commit_id'] + df['commit_title'] + df['commit_comments']
            total_tokens = 0
            for i in range(len(df.index)):
                text = df['content'][i]
                token_len = num_tokens_from_string(text, encode)
                total_tokens = total_tokens + token_len
            total_cost = get_embedding_cost(total_tokens, cost)
            return total_cost
        except Exception as e:
            print('Error in get_total_embeddings_cost: {}'.format(e)) 

def create_commit_hist_table(pg_notice):
    try:
        cur = conn.cursor()
        sql = '''
            DROP TABLE IF EXISTS commit_history;
        
            CREATE TABLE commit_history (
                id BIGINT PRIMARY KEY,
                author TEXT NOT NULL,
                commit_time TIMESTAMPTZ NOT NULL,
                commit_id TEXT NOT NULL,
                commit_title TEXT NOT NULL,
                commit_comments TEXT     
            );
        '''
        cur.execute(sql)
        cur.close()
        conn.commit()
        pg_notice()
        return True
    except Exception as e:
        print('Error in create_commit_hist_table: {}'.format(e)) 
        
def read_commit_hist(cols):
    try:
        df = pd.read_csv('data/input/commit_history.csv', names=cols)
        return df
    except Exception as e:
        print('Error in read_commit_hist: {}'.format(e))
        
def write_commit_hist(df, pg_notice):
    try:
        table = 'commit_history'
        tuples = [tuple(rec) for rec in df.to_numpy()]
        cols = ','.join(list(df.columns)) 
        # SQL query to execute 
        query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
        cur = conn.cursor() 
        try: 
            extras.execute_values(cur, query, tuples) 
            pg_notice()
            conn.commit() 
        except (Exception, pg.DatabaseError) as error: 
            print("Error: %s" % error) 
            conn.rollback() 
            cur.close() 
            return False
        print("the dataframe is inserted") 
        cur.close()
        return True
    except Exception as e:
        print('Error in read_commit_hist: {}'.format(e))

def fetch_ch_ids():
    cur = conn.cursor()
    query = '''
        SELECT array_to_json(array_agg(id)) from
            (SELECT id FROM commit_history) sq
    '''
    cur.execute(query)
    return cur.fetchall()[0][0]

def create_vector_table(pg_notice):
    try:
        cur = conn.cursor()
        sql = '''
            DROP TABLE IF EXISTS commit_history_vector;
        
            CREATE TABLE commit_history_vector (
                id BIGINT PRIMARY KEY,
                embedding vector(1536) -- the vector type comes from the pgvector extension
            );
        '''
        cur.execute(sql)
        conn.commit()
        pg_notice()
    except Exception as e:
        print('Error in create_vector_table: {}'.format(e))    

def ch_vector_inserts(pg_notice, embed_model, commit_history_id):
    try:
        cur = conn.cursor()
        sql = '''
            INSERT INTO commit_history_vector (id, embedding) 
                SELECT 
                    id,
                    openai_embed (
                        %(model)s,
                        -- create a single text string representation of the commit
                        author || commit_time::text || commit_id || commit_title || commit_comments,
                        -- pass api key
                        %(key)s
                    ) as embedding
                FROM commit_history where id = %(id)s
        '''
        cur.execute(sql, {'model': embed_model, 'key': open_ai_key, 'id': commit_history_id})
        pg_notice()
        conn.commit()
        cur.close()
        return True
    except Exception as e:
        print('Error in ch_vector_inserts: {}'.format(e)) 

def main():    
    ClassObjs = Openai()
    params = ClassObjs.get_params()
    notices = ClassObjs.get_notices
    create_commit_hist_table(notices)
    commit_hist_data = read_commit_hist(params['ch_cols'])
    write_commit_hist(commit_hist_data, notices) 
    
    # BEFORE EMBEDDING CHECK COST!!!! -- if coolio, then uncomment ch_vector_table :)
    embedding_cost = ClassObjs.get_total_embeddings_cost(commit_hist_data, params['encoding'], params['token_cost'])
    print('embed cost: {}'.format(np.round(embedding_cost, 4)))
    
    # seems faster to iterate over ids versus import all together
    create_vector_table(notices)
    ids = fetch_ch_ids()
    insert_count = 0
    for id in ids:
        ch_vector_inserts(notices, params['model'], id)   
        insert_count += 1 
        if insert_count % 100 == 0:
            print('vectors inserted: {}'.format(insert_count))
    return commit_hist_data, ids

ret = main()
